In [1]:
import jsonlines
from tqdm import tqdm
import os

In [2]:
task_list = ['CHIP-CDEE', 'CMeEE-V2', 'CHIP-CTC', 'IMCS-V2-DAC', 'CHIP-STS', 'IMCS-V2-MRG', 'MedDG', 'CHIP-CDN', 'IMCS-V2-NER', 'CHIP-MDCFNPC', 'KUAKE-QTR', 'KUAKE-QIC', 'CMeIE', 'IMCS-V2-SR', 'KUAKE-QQR', 'KUAKE-IR']
main_path = "./"

In [3]:
def read_data(data_path):

    data = []

    with jsonlines.open(data_path, "r") as f:
        for meta_data in f:
            data.append(meta_data)

    return data

In [4]:
def save_data(data_path, data):
    # write all_data list to a new jsonl
    with jsonlines.open(data_path, "w") as w:
        for meta_data in data:
            w.write(meta_data)

In [5]:
def extract_data(data):
# {"sample_id": data_jsonline}
    data_dict = {}

    for meta_data in tqdm(data):

        if meta_data['sample_id'] in data_dict.keys():
            #print(data_dict[meta_data['sample_id']]['task_dataset'])
            #print(meta_data['task_dataset'])
            #print('\n')
            if data_dict[meta_data['sample_id']]['task_dataset'] == meta_data['task_dataset']:
                print(meta_data['task_dataset'])
                print(meta_data['sample_id'])
                break

        data_dict[meta_data['sample_id']] = meta_data

    return data_dict

In [6]:
def extract_data_new(data):
# [("sample_id", "task_dataset"), ...]
    data_list = []

    for meta_data in tqdm(data):

        data_list.append((meta_data['sample_id'], meta_data['task_dataset']))

    return data_list

In [7]:
raw_test = extract_data_new(read_data("testB.json"))
#raw_index = list(raw_test.keys())

100%|██████████| 10072/10072 [00:00<00:00, 1457428.75it/s]


In [8]:
res_dict = {}
# ensemble all test results
for task in task_list:

    task_path = os.path.join(main_path, task)
    task_path = os.path.join(task_path, "test_predictions.json")
    task_dict = extract_data(read_data(task_path))

    res_dict[task] = task_dict

100%|██████████| 600/600 [00:00<00:00, 1451316.26it/s]


In [9]:
# whether has the same test samples
#assert len(res_dict) == len(raw_test)
print(len(raw_test))

10072


In [10]:
results = []
# search the test sample from all test results
for sample_id, task_dataset in raw_test:
    results.append(res_dict[task_dataset][sample_id])

save_data("test_predictions.json", results)